# Package and Initial conditions setup

In [ ]:
from projectq import MainEngine  # import the main compiler engine
from projectq.ops import H, Swap, C, Tensor, Toffoli, QFT, All, X, Z, Measure, CNOT, StatePreparation, Rx, Ry, Rz  # import the operations we want to perform (Hadamard and measurement)
from projectq.ops import TimeEvolution
from projectq.meta import Control, Dagger, Loop
from projectq.backends import CircuitDrawer 
from projectq.types._qubit import Qureg

import matplotlib
#matplotlib.use("Agg")
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.path as path
import matplotlib.animation as animation
from IPython.display import HTML

import numpy as np
import scipy
import random
import bisect
from tqdm import tqdm_notebook as tqdm

import projectq.setups.decompositions

# ONLY USE DECOMPOSITION rule_set
rule_set = projectq.cengines.DecompositionRuleSet(
    modules=[projectq.setups.decompositions])
engines = [projectq.cengines.AutoReplacer(rule_set)]

#from projectq.ops import QubitOperator
from openfermion.ops import QubitOperator #Note this is another operator than the projectq operator

from openfermion.ops import FermionOperator  #the creation annihilation operators
from openfermion.utils import hermitian_conjugated
from openfermion.transforms import jordan_wigner, get_sparse_operator
from openfermion.utils import hermitian_conjugated, eigenspectrum, trotterize_exp_qubop_to_qasm as trot


eng = MainEngine(engine_list=engines)
precision_evolution_time = 1
evolution_times = [600]
N_lattice = 8
delta = 0.5

State_string = '|01001001>'

#number of simulations we do to average
N_average = 1

random.seed(102934)

# Initial State Preparation and desired Operator

In [ ]:
#with this function one can just setup the desired state more easily by just stating the desired states.
#Note that the state |001>, that intuitively corresponds to the value |1> in decimal state is an X gate on the first qubit (i.e. X | q[0]).
#And the state |100>, intuitively corresponds to the value |4> in the decimal state and is an x gate on the third qubit (i.e. X | q[2]).
def String_State_Tuple(state_string):
    """As the input string we want e.g. '0.5|0010> + 0.5|0011> + 0.5**0.5 |1000>' this will be converted 
    to the decimal representation '0.5 |2> + 0.5|3>  0.5**0.5 |8>' which will then be converted into the list
    [0, 0, 0.5, 0.5, 0, 0, 0, 0, 0.5**0.5, 0, 0, 0, 0, 0, 0, 0]"""

    #We add a space before and after every state
    state_string = state_string.replace('|', ' |')
    state_string = state_string.replace('>', '> ')

    #We look for the brackets in the string
    state_tuple_list = []
    split_string = state_string.split()

    for i in range(len(split_string)):  #split up the string
        elem = split_string[i]
        elem_previous = split_string[i - 1]
        pos1 = elem.find('|')  #search for a state
        if (pos1 == -1):  #we did not find a state in this string
            continue
        pos2 = elem.find('>')
        qubit_value = 0
        if (pos1 != -1 and pos2 == (len(elem) - 1)
            ):  #i.e. we found a '|' somewhere and '>' is on the last position
            qubit_string = elem[(pos1 + 1):pos2]
            assert len(
                qubit_string
            ) == N_lattice  #Check if the prepared number of qubits is equal to that given in the string
            for i in range(len(qubit_string)):
                if (qubit_string[-i - 1] == '1'):
                    qubit_value += 2**i
            #there are two possibilities on the input, either there was a space before or not
            #if there a space we have the amplitude in the element in front of the state_string
            if (pos1 == 0):
                try:
                    qubit_amplitude = eval(elem_previous)
                except SyntaxError:  #then we don't have a wave number in front and the wave number is 1
                    if (
                            len(split_string) > 1
                    ):  #if there is more than one element the wavefunction is not normalized
                        print('Error Wave function is not properly defined')
                        return
                    qubit_amplitude = 1.0
            #if there was no space the amplitude is in front of the ket
            else:
                qubit_amplitude = eval(elem[:pos1])
        state_tuple = (qubit_value, qubit_amplitude)
        state_tuple_list.append(state_tuple)

    return (state_tuple_list)


def Check_normalization(state_list):
    assert type(state_list) is list  #check if argument is a list

    probability_sum = 0
    for i in range(len(state_list)):
        assert type(state_list[i]) is tuple  #check if argument is a tuple
        probability_sum += state_list[i][1]**2

    #check difference with one
    diff = abs(probability_sum - 1.0)

    if diff < 10**-5:
        return True
    else:
        return False

def String_Tuple_ToState(tuple_string):
    #Here we create a list with acculaed like [0,0,0,0.5,0.5,0.5,0] etc. for the state preparation and apply the state preparation
    assert type(tuple_string) is list
    #assert type(qubits) is Qureg

    accu_list = []

    for i in range(2**N_lattice):
        for k in range(len(tuple_string)):
            if (tuple_string[k][0] == i):
                accu_list.append(tuple_string[k][1])
                break
        else:
            accu_list.append(0)

    assert np.linalg.norm(accu_list) == 1.0  #check if list is normalized
    assert len(accu_list) == 2**N_lattice  #Check if we did not append anything extra by accident

    return accu_list

def String_to_vector(state_string):
    acculist = String_Tuple_ToState(String_State_Tuple(state_string))
    
    return acculist

def String_to_State(state_string, qubits):
  
    acculist = String_to_vector(state_string)  
    
    #Now we perform the state preparation
    eng.flush()

    #Better to not use statepreparation because it optimizes poorly
    
    #However since we've included only the needed gate sets now we can
    StatePreparation(acculist) | qubits

    #We use the backend cheat wavefunction setup. 
    #Note that the backend cheat function does not need to initialize on the all zero state, but works on any state
    #However the cheat function does not work when we have multiple qubits so in our program it will not work
    #eng.backend.set_wavefunction(acculist, qubits)
    
    #set the wave function
    eng.flush()


def State_to_String(qubits):
    '''In this function we will return a string with the state from the regular qubit input to help visualizing the state.'''
    state_string = ''

    assert type(qubits) is Qureg

    no_qubits = len(qubits)  #number of qubits

    #We create the list with the wavefunction
    wave_round = []  #list with rounded values of the coefficients of the qubits

    #We will use the get_amplitude backend function, for this we need to input the desired state
    for i in range(2**no_qubits):
        value_bin = bin(i)  #returns '0b10101'
        value_bin = value_bin[2:]  #Gets rid of '0b'
        while (len(value_bin) < no_qubits):
            value_bin = '0' + value_bin
        assert len(value_bin) == no_qubits
        #because the backend.get functions see the binary numbers inverted i.e. '0001' is 8 instead of our '1000' is 8 we invert the string
        eng.flush()
        probability = eng.backend.get_probability(
            value_bin[::-1], qubits
        )  #get_amplitude only works on calling all qubits, so this scheme does not work
        prob_round = np.round(probability, 8)
        wave_round.append(prob_round)

    #We loop over the entries of the wave function
    for i in range(len(wave_round)):
        if (wave_round[i] > 0.0):
            #We bring the state value in decimal to a binary value
            value_bin = bin(i)  #returns '0b10101'
            value_bin = value_bin[2:]
            while (len(value_bin) < no_qubits):
                value_bin = '0' + value_bin

            #Write in bracket notation
            value_bin = '|' + value_bin + '>'

            #Write rounded coefficient in front with plus sign behind
            value_string = str(wave_round[i]) + value_bin + ' + '

            state_string += value_string

    state_string = state_string[:-3]  #Remove the last ' + '
    return state_string


def State_to_prob(qubits):
    '''In this function we will return a list that consists of the probabilites of all the different states'''
    assert type(qubits) is Qureg

    no_qubits = len(qubits)  #number of qubits

    prob_round_list = []

    for i in range(2**no_qubits):
        value_bin = bin(i)  #returns '0b10101'
        value_bin = value_bin[2:]  #Gets rid of '0b'
        while (len(value_bin) < no_qubits):
            value_bin = '0' + value_bin
        assert len(value_bin) == no_qubits
        #because the backend.get functions see the binary numbers inverted i.e. '0001' is 8 instead of our '1000' is 8 we invert the string
        eng.flush()
        probability = eng.backend.get_probability(
            value_bin[::-1], qubits
        )  #get_amplitude only works on calling all qubits, so this scheme does not work
        prob_round = np.round(probability, 8)
        prob_round_list.append(prob_round)

    return prob_round_list

def random_supersym_state(num_part = 2):
    '''This function outputs a random state in which we do not place two particles next to eachother
    because this will not time evolve so the situation is not interesting. We do this by generating a random string and 
    if there are particles next to eachother we discard it. This means we cannot place more than 4 particles. 
    Also 1 particle is never interesting, so we want 2 or 3 particles'''
    
    #Create random number of particles and list
#     num_part = random.randrange(2,4)
    lam_list = [1 for i in range(num_part)] + [0 for i in range(N_lattice - num_part)]
    
    #Now shuffle the list and check if it suffices 
    while True:
        random.shuffle(lam_list)
        random.shuffle(lam_list)
        shuf_sum = 0
        
        for i in range(len(lam_list) - 1):
            #If the product of an element with the next is 0 for every element 
            #we have that no 1's and 0's are next to eachother
            shuf_sum += lam_list[i] * lam_list[i+1]
        
        if shuf_sum == 0:
            #we want to output a string
            B = ''
            for elem in lam_list:
                B += str(elem)
            
            B = '|' + B + '>'
                
            return B

# Encoding the Hamiltonian

In this section we will define the Hamiltonian $H = \sum_i J_\bot (\hat{s}_i^x \hat{s}_{i+1}^x + \hat{s}_{i}^y \hat{s}_{i+1}^y) + J_z \hat{s}_{i}^z \hat{s}_{i+1}^z + h_i \hat{s}_{i}^z $, where $h_i$ is drawn uniformly from the interval $[-h,h]$. This Hamiltonian is already in the pauli spin operators setup. 

In [ ]:
def XXZ_sum_terms_Pauli_random_strength(i,
                                        spin_operator=1,
                                        spin_z_operator=1,
                                        field_stregth=1):
    '''This function writes one term of the total sum of the Hamiltonian. 
        This will be used in the time evolution operator of projectq. 
        We use open boundary conditions so whenever i = (N_lattice - 1) we say (i+1) = 0 '''

    spin_operator = float(spin_operator)
    spin_z_operator = float(spin_z_operator)
    field_stregth = float(field_stregth)

    first_qubit = i

    if i == N_lattice - 1:
        second_qubit = 0

    else:
        second_qubit = i + 1

    local_strength = (
        (random.random() - 0.5) *
        2) * field_stregth  #The first term is a random number in [-1,1]

    term_X = QubitOperator(f'X{first_qubit} X{second_qubit}', spin_operator)
    term_Y = QubitOperator(f'Y{first_qubit} Y{second_qubit}', spin_operator)
    term_Z = QubitOperator(f'Z{first_qubit} Z{second_qubit}', spin_z_operator)
    local_term = QubitOperator(f'Z{i}', local_strength)

    term_sum = term_X + term_Y + term_Z + local_term

    return term_sum


def XXZ_Hamiltonian(spin_operator=1, spin_z_operator=1, field_stregth=1):

    ham = QubitOperator()  #Empty qubit operator

    for i in range(N_lattice):
        term = XXZ_sum_terms_Pauli_random_strength(
            i, spin_operator, spin_z_operator, field_stregth)
        ham += term

    assert type(ham) == QubitOperator

    return ham

Here we will define the type III supersymmetric Hamiltonian

We take the Jordan-Wigner transformation of the Hamiltonian. Then we use the openfermion command to generate the trotterized version of this Hamiltonian. This gives us the operator $e^{-iH \delta t}$, for some timestep $\delta t$. This operator applied to a state time evolves the state under the Hamiltonian. \\

\begin{align}
    Q^\dagger = \sum_{i=0}^{N-1} \lambda_i (1-n_{i-1}) c_i^\dagger (1-n_{i+1})
\end{align}

$\lambda_i$ in interval around 1. $H = \{Q^\dagger, Q\}$

In [ ]:
def CountingOperator(i):  
    '''This creates cdagger c at some position i and takes the boundary conditions into account'''

    #Set up boundary conditions, we say that the positions outside the chain are 'filled' 
    #and we return an empty operator that is like the identity
    if i == -1:  
        return FermionOperator()  
    
    if i == N_lattice: 
        return FermionOperator()
    
    #For the lattice points on the chain we create the counting operator with the creation annihilation operator
    i_string_dagger = str(i) + '^ '  
    i_string = str(i)  #This creates 'i'
    total_string = i_string_dagger + i_string  #'i^ i'
    c_dagger_c = FermionOperator(total_string)  #This makes cdagger_i c_i

    return c_dagger_c

def Q_dagger_term(i):  #This creates all the individual terms of the sum
    '''This creates all the individual terms of the sum that we use for the Hamiltonian. '''  
    
    #Create creation operator
    i_dagger = str(i) + '^ '  #'i^ '
    Identity = FermionOperator('')

    #We create the term in the sum of Q^Dagger
    term = (Identity - CountingOperator(i - 1)) * FermionOperator(i_dagger) * (Identity - CountingOperator(i + 1)) 
    
    return term

# def supersymmetric_Hamiltonian(width_interval = 0, lambda_pos = [0 for i in range(N_lattice)]):
#     '''This function generates the Hamiltonian in the second quantizes version and it takes into account the 
#     random values lambda_i that we multiply with some of the terms. It outputs the jordan wigner transposed Hamiltonian
    
#     It takes as an input the width of the 
#     interval from which we will randomly draw the values for labda, and it takes a list with the positions where
#     we want lambda not equal to 1. The list has values 0 or 1 where 0 indicates we do not take a random lambda and 
#     1 indicates we do'''
    
# #     #Assert that the position list has length N_lattice and it consists of integers 0 or 1
# #     assert len(lambda_pos) == N_lattice
    
# #     for elem in lambda_pos:
# #         assert type(elem) == int
# #         assert (elem == 0 or elem == 1)
        
#     #Create empty operator
#     Q_dagger = FermionOperator()

#     #Now we create our Operator Q
#     for i in range(N_lattice):
#         #If the lambda_pos value is 1 we create a random value in [1-w, 1+w]
#         if lambda_pos[i] == 1:
#             random_lambda = 1 + ((random.random() - 0.5) * 2) * width_interval 
        
#         else:
#             random_lambda = 1
        
#         Q_dagger += random_lambda * Q_dagger_term(i)

#     #We take the Hermitian Conjugate of this operator
#     Q = hermitian_conjugated(Q_dagger)  

#     #This will be our Hamiltonian
#     Hamiltonian = Q * Q_dagger + Q_dagger * Q  

#     #We take the jordan-wigner transformation
#     Hamiltonian_jw = jordan_wigner(Hamiltonian)  

#     #Return the jordan-wigner transformed Hamiltonian
#     return Hamiltonian_jw


def generate_hamiltonian_trot_supersym(Hamiltonian_jw):
    '''This a generator of the code in the QASM language that we make into a list and 
    have to be applied in succession on the quantum computer'''
    Hamiltonian_trot = list(
        trot(Hamiltonian_jw, evolution_time=precision_evolution_time))

    return Hamiltonian_trot

# Time Evolution of Hamiltonian

In [ ]:
#Since openfermion generates code in the QASM format I need to change it a bit to get code in the ProjectQ format.
#This command takes as input the lines created by the generator and performs the operations on the qubits in projectq
def QASM_to_ProjectQ(string, qubits):
    #eng.flush()
    if string[:4] == 'CNOT':  #we perform a CNOT
        position_1 = int(string[5])
        position_2 = int(string[7])
        #print('time for CNOT of ', State_to_String(qubits))
        CNOT | (qubits[position_1], qubits[position_2])
        #print('CNOT done')

    if string[:
              1] == 'H':  #If we read out a hadamard we perform itWe perform the Hadamard
        #print('time for Hadamard', State_to_String(qubits))
        H | qubits[int(string[2])]
        #print('Hadamard done')

    if string[:1] == 'R':  #We perform a rotation in x,y or z
        #we check where the second ' ' is and take the numbers until there as the angle
        axes = string[1]
        Operation = string[:2]
        angle = float(string[3:string.find(' ', 3)])
        #print('time for Rotation in '+axes+'direction', State_to_String(qubits))
        eval(Operation)(angle) | qubits[int(string[-1])]
        #print('Rotation done')
        
#This command time evolves some state with the Hamiltonian for some evolution time
def time_evolve(qubits, Hamiltonian_trot):
    #Hamiltonian_trot_QASM = io.StringIO(Hamiltonian_trot_to_list(Hamiltonian_trot))
    for line in Hamiltonian_trot:
        QASM_to_ProjectQ(line, qubits)

# Extracting the wave function from the state

In [ ]:
def cheat_order_qubits():
    '''This function takes no input as it works on all qubits. It returns the wave function of the quantum state 
    where the order of the qubits is from the 0th qubit to the final qubit. We need to do this
    because projectq can have a different order for the qubits. The wave function can be used 
    later to set up a new state with the same wave function or it can be used to calculate the 
    partial traces.'''

    #We flush and generate the entire wave function of all the qubits
    eng.flush()
    cheat_wave = eng.backend.cheat()

    #Check how many qubits we have and determine the mapping order
    num_qubits = len(cheat_wave[0])
    order_qubits = cheat_wave[0]
    assert type(order_qubits) == dict
    
    #Note that even though the number of qubits we have goes from 1 to num_qubits
    #this doesn't mean that they are placed in the dict from 0 to num_qubits because
    #in projectq, if we create and destroy and then create a qubit it will live at 
    #place 1 in the dict. So we search for the lowest number in the dict and then check the qubits
    
    #Generate 
    lst_order = list(order_qubits) 
    min_val = np.min(lst_order)
    min_loc = lst_order.index(min_val)
    
    #This means the dict starts at the number min_val, now we just make our list 
    transpose_order = [order_qubits[i + min_val] for i in range(num_qubits)]

    #Now we construct how we want to swap our qubits e.g. order [0,3,2,1] -> [0,1,2,3],
    #note that this is just sending the location to its value.

    #Now we access the wave function and check if its length corresponds to our number of qubits
    wave_function = cheat_wave[1]
    assert len(wave_function) == 2**num_qubits

    #Reshape the wave function to per qubit level
    wave_qubits = np.reshape(wave_function, [2 for i in range(num_qubits)])

    #Now we transpose the positions
    ordered_wave_qubits = np.transpose(wave_qubits, transpose_order)

    #Reshape to the multi qubit state
    ordered_wave = np.reshape(ordered_wave_qubits, [2**num_qubits])

    return ordered_wave


def reverse_qubits(wave_function):
    '''This function takes as an input the full wave function and reverses the order of the qubits
    We sometimes need this because for the wave |abc> projectq views the a as the 3rd qubit. So 
    when calculating the partial trace we need to reverse the order of the qubits.
    
    We output the full wave function of the reversed qubits'''

    #Get the number of qubits from the dimension of the wave function
    num_qubits = np.log2(len(wave_function))
    assert num_qubits.is_integer()
    num_qubits = int(num_qubits)

    #Reshape the wave function to per qubit level
    wave_qubit = np.reshape(wave_function, [2 for i in range(num_qubits)])

    #Reverse the order
    rev_order = [i for i in range(num_qubits)]
    rev_order.reverse()

    #Transpose the qubits
    rev_wave_qubit = np.transpose(wave_qubit, rev_order)

    #Reshape to the multi qubit state
    rev_wave = np.reshape(rev_wave_qubit, [2**num_qubits])

    return rev_wave


def check_wave_function_size(wave_function, size_lst):
    assert type(size_lst) == list
    assert len(size_lst) == 4

    #Assert that our sizes are correct
    num_qubits = np.log2(len(wave_function))
    assert num_qubits.is_integer()
    num_qubits = int(num_qubits)
    assert np.prod(size_lst) == 2**num_qubits

    return True

# Calculation of the entanglement entropy and reduced state entropies

In [ ]:
def entanglement_entropy(sizeA, sizeB):
    assert type(sizeA) == int
    assert type(sizeB) == int

    #Get wave function
    wave_function = cheat_order_qubits()
    assert sizeA * sizeB == len(wave_function)

    #Access the reversed wave function
    wave_rev = reverse_qubits(wave_function)

    #Now we reshape the wave function to calculate the partial traces
    wave_reshaped = np.reshape(wave_rev, [sizeA, sizeB])

    rho = wave_reshaped @ wave_reshaped.conj().transpose()
    Entropy = -np.log2(np.trace(rho @ rho))

    assert np.isreal(Entropy)
    Entropy = np.real(Entropy)

    return Entropy


def Entropy_R(wave_function, size_lst):

    assert check_wave_function_size(wave_function, size_lst)

    #Access the reversed wave function
    wave_rev = reverse_qubits(wave_function)

    #Now we reshape the wave function to calculate the partial traces
    wave_reshaped = np.reshape(wave_rev, size_lst)

    #Calculate sizes of subsystems
    size_1 = size_lst[0]
    size_2 = size_lst[1] * size_lst[2] * size_lst[3]

    #Calculate Entropy R
    wave_state = np.transpose(wave_reshaped, (0, 1, 2, 3))
    wave_partial = wave_state.reshape(size_1, size_2)
    rho = wave_partial @ wave_partial.conj().transpose()
    Entropy = -np.log2(np.trace(rho @ rho))

    return Entropy


def Entropy_C(wave_function, size_lst):

    assert check_wave_function_size(wave_function, size_lst)

    #Access the reversed wave function
    wave_rev = reverse_qubits(wave_function)

    #Now we reshape the wave function to calculate the partial traces
    wave_reshaped = np.reshape(wave_rev, size_lst)

    #Calculate sizes of subsystems
    size_1 = size_lst[1]
    size_2 = size_lst[0] * size_lst[2] * size_lst[3]

    #Calculate Entropy R
    wave_state = np.transpose(wave_reshaped, (1, 0, 2, 3))
    wave_partial = wave_state.reshape(size_1, size_2)
    rho = wave_partial @ wave_partial.conj().transpose()
    Entropy = -np.log2(np.trace(rho @ rho))

    return Entropy


def Entropy_D(wave_function, size_lst):

    assert check_wave_function_size(wave_function, size_lst)

    #Access the reversed wave function
    wave_rev = reverse_qubits(wave_function)

    #Now we reshape the wave function to calculate the partial traces
    wave_reshaped = np.reshape(wave_rev, size_lst)

    #Calculate sizes of subsystems
    size_1 = size_lst[2]
    size_2 = size_lst[0] * size_lst[1] * size_lst[3]

    #Calculate Entropy R
    wave_state = np.transpose(wave_reshaped, (2, 1, 0, 3))
    wave_partial = wave_state.reshape(size_1, size_2)
    rho = wave_partial @ wave_partial.conj().transpose()
    Entropy = -np.log2(np.trace(rho @ rho))

    return Entropy


def Entropy_B(wave_function, size_lst):

    assert check_wave_function_size(wave_function, size_lst)

    #Access the reversed wave function
    wave_rev = reverse_qubits(wave_function)

    #Now we reshape the wave function to calculate the partial traces
    wave_reshaped = np.reshape(wave_rev, size_lst)

    #Calculate sizes of subsystems
    size_1 = size_lst[3]
    size_2 = size_lst[0] * size_lst[1] * size_lst[2]

    #Calculate Entropy R
    wave_state = np.transpose(wave_reshaped, (3, 1, 2, 0))
    wave_partial = wave_state.reshape(size_1, size_2)
    rho = wave_partial @ wave_partial.conj().transpose()
    Entropy = -np.log2(np.trace(rho @ rho))

    return Entropy


def Entropy_RC(wave_function, size_lst):

    assert check_wave_function_size(wave_function, size_lst)

    #Access the reversed wave function
    wave_rev = reverse_qubits(wave_function)

    #Now we reshape the wave function to calculate the partial traces
    wave_reshaped = np.reshape(wave_rev, size_lst)

    #Calculate sizes of subsystems
    size_1 = size_lst[0] * size_lst[1]
    size_2 = size_lst[2] * size_lst[3]

    #Calculate Entropy R
    wave_state = np.transpose(wave_reshaped, (0, 1, 2, 3))
    wave_partial = wave_state.reshape(size_1, size_2)
    rho = wave_partial @ wave_partial.conj().transpose()
    Entropy = -np.log2(np.trace(rho @ rho))

    return Entropy


def Entropy_CR(wave_function, size_lst):
    return Entropy_RC(wave_function, size_lst)


def Entropy_RD(wave_function, size_lst):

    assert check_wave_function_size(wave_function, size_lst)

    #Access the reversed wave function
    wave_rev = reverse_qubits(wave_function)

    #Now we reshape the wave function to calculate the partial traces
    wave_reshaped = np.reshape(wave_rev, size_lst)

    #Calculate sizes of subsystems
    size_1 = size_lst[0] * size_lst[2]
    size_2 = size_lst[1] * size_lst[3]

    #Calculate Entropy R
    wave_state = np.transpose(wave_reshaped, (0, 2, 1, 3))
    wave_partial = wave_state.reshape(size_1, size_2)
    rho = wave_partial @ wave_partial.conj().transpose()
    Entropy = -np.log2(np.trace(rho @ rho))

    return Entropy


def Entropy_DR(wave_function, size_lst):
    return Entropy_RD(wave_function, size_lst)


def Entropy_RB(wave_function, size_lst):

    assert check_wave_function_size(wave_function, size_lst)

    #Access the reversed wave function
    wave_rev = reverse_qubits(wave_function)

    #Now we reshape the wave function to calculate the partial traces
    wave_reshaped = np.reshape(wave_rev, size_lst)

    #Calculate sizes of subsystems
    size_1 = size_lst[0] * size_lst[3]
    size_2 = size_lst[1] * size_lst[2]

    #Calculate Entropy R
    wave_state = np.transpose(wave_reshaped, (0, 3, 2, 1))
    wave_partial = wave_state.reshape(size_1, size_2)
    rho = wave_partial @ wave_partial.conj().transpose()
    Entropy = -np.log2(np.trace(rho @ rho))

    return Entropy


def Entropy_BR(wave_function, size_lst):
    return Entropy_RB(wave_function, size_lst)


def Entropy_CD(wave_function, size_lst):

    assert check_wave_function_size(wave_function, size_lst)

    #Access the reversed wave function
    wave_rev = reverse_qubits(wave_function)

    #Now we reshape the wave function to calculate the partial traces
    wave_reshaped = np.reshape(wave_rev, size_lst)

    #Calculate sizes of subsystems
    size_1 = size_lst[1] * size_lst[2]
    size_2 = size_lst[0] * size_lst[3]

    #Calculate Entropy R
    wave_state = np.transpose(wave_reshaped, (2, 1, 0, 3))
    wave_partial = wave_state.reshape(size_1, size_2)
    rho = wave_partial @ wave_partial.conj().transpose()
    Entropy = -np.log2(np.trace(rho @ rho))

    return Entropy


def Entropy_DC(wave_function, size_lst):
    return Entropy_CD(wave_function, size_lst)


def Entropy_CB(wave_function, size_lst):

    assert check_wave_function_size(wave_function, size_lst)

    #Access the reversed wave function
    wave_rev = reverse_qubits(wave_function)

    #Now we reshape the wave function to calculate the partial traces
    wave_reshaped = np.reshape(wave_rev, size_lst)

    #Calculate sizes of subsystems
    size_1 = size_lst[1] * size_lst[3]
    size_2 = size_lst[0] * size_lst[2]

    #Calculate Entropy R
    wave_state = np.transpose(wave_reshaped, (2, 3, 0, 1))
    wave_partial = wave_state.reshape(size_1, size_2)
    rho = wave_partial @ wave_partial.conj().transpose()
    Entropy = -np.log2(np.trace(rho @ rho))

    return Entropy


def Entropy_BC(wave_function, size_lst):
    return Entropy_CB(wave_function, size_lst)


def Entropy_BD(wave_function, size_lst):

    assert check_wave_function_size(wave_function, size_lst)

    #Access the reversed wave function
    wave_rev = reverse_qubits(wave_function)

    #Now we reshape the wave function to calculate the partial traces
    wave_reshaped = np.reshape(wave_rev, size_lst)

    #Calculate sizes of subsystems
    size_1 = size_lst[3] * size_lst[2]
    size_2 = size_lst[0] * size_lst[1]

    #Calculate Entropy R
    wave_state = np.transpose(wave_reshaped, (2, 3, 0, 1))
    wave_partial = wave_state.reshape(size_1, size_2)
    rho = wave_partial @ wave_partial.conj().transpose()
    Entropy = -np.log2(np.trace(rho @ rho))

    return Entropy


def Entropy_DB(wave_function, size_lst):
    return Entropy_BD(wave_function, size_lst)


def Entropy_RCD(wave_function, size_lst):

    assert check_wave_function_size(wave_function, size_lst)

    #Access the reversed wave function
    wave_rev = reverse_qubits(wave_function)

    #Now we reshape the wave function to calculate the partial traces
    wave_reshaped = np.reshape(wave_rev, size_lst)

    #Calculate sizes of subsystems
    size_1 = size_lst[0] * size_lst[1] * size_lst[2]
    size_2 = size_lst[3]

    #Calculate Entropy R
    wave_state = np.transpose(wave_reshaped, (0, 1, 2, 3))
    wave_partial = wave_state.reshape(size_1, size_2)
    rho = wave_partial @ wave_partial.conj().transpose()
    Entropy = -np.log2(np.trace(rho @ rho))

    return Entropy


def Entropy_RCB(wave_function, size_lst):

    assert check_wave_function_size(wave_function, size_lst)

    #Access the reversed wave function
    wave_rev = reverse_qubits(wave_function)

    #Now we reshape the wave function to calculate the partial traces
    wave_reshaped = np.reshape(wave_rev, size_lst)

    #Calculate sizes of subsystems
    size_1 = size_lst[0] * size_lst[1] * size_lst[3]
    size_2 = size_lst[2]

    #Calculate Entropy R
    wave_state = np.transpose(wave_reshaped, (0, 1, 3, 2))
    wave_partial = wave_state.reshape(size_1, size_2)
    rho = wave_partial @ wave_partial.conj().transpose()
    Entropy = -np.log2(np.trace(rho @ rho))

    return Entropy


def Entropy_RBC(wave_function, size_lst):
    return Entropy_RCB(wave_function, size_lst)


def Entropy_BRC(wave_function, size_lst):
    return Entropy_RCB(wave_function, size_lst)


def Entropy_BCR(wave_function, size_lst):
    return Entropy_RCB(wave_function, size_lst)


def Entropy_CBR(wave_function, size_lst):
    return Entropy_RCB(wave_function, size_lst)


def Entropy_CRB(wave_function, size_lst):
    return Entropy_RCB(wave_function, size_lst)


def Entropy_RDB(wave_function, size_lst):

    assert check_wave_function_size(wave_function, size_lst)

    #Access the reversed wave function
    wave_rev = reverse_qubits(wave_function)

    #Now we reshape the wave function to calculate the partial traces
    wave_reshaped = np.reshape(wave_rev, size_lst)

    #Calculate sizes of subsystems
    size_1 = size_lst[0] * size_lst[2] * size_lst[3]
    size_2 = size_lst[1]

    #Calculate Entropy R
    wave_state = np.transpose(wave_reshaped, (0, 3, 2, 1))
    wave_partial = wave_state.reshape(size_1, size_2)
    rho = wave_partial @ wave_partial.conj().transpose()
    Entropy = -np.log2(np.trace(rho @ rho))

    return Entropy


def Entropy_RBD(wave_function, size_lst):
    return Entropy_RDB(wave_function, size_lst)


def Entropy_BDR(wave_function, size_lst):
    return Entropy_RDB(wave_function, size_lst)


def Entropy_BRD(wave_function, size_lst):
    return Entropy_RDB(wave_function, size_lst)


def Entropy_DBR(wave_function, size_lst):
    return Entropy_RDB(wave_function, size_lst)


def Entropy_DRB(wave_function, size_lst):
    return Entropy_RDB(wave_function, size_lst)


def Entropy_CDB(wave_function, size_lst):

    assert check_wave_function_size(wave_function, size_lst)

    #Access the reversed wave function
    wave_rev = reverse_qubits(wave_function)

    #Now we reshape the wave function to calculate the partial traces
    wave_reshaped = np.reshape(wave_rev, size_lst)

    #Calculate sizes of subsystems
    size_1 = size_lst[1] * size_lst[2] * size_lst[3]
    size_2 = size_lst[0]

    #Calculate Entropy R
    wave_state = np.transpose(wave_reshaped, (3, 1, 2, 0))
    wave_partial = wave_state.reshape(size_1, size_2)
    rho = wave_partial @ wave_partial.conj().transpose()
    Entropy = -np.log2(np.trace(rho @ rho))

    return Entropy


def Entropy_CBD(wave_function, size_lst):
    return Entropy_CDB(wave_function, size_lst)


def Entropy_BCD(wave_function, size_lst):
    return Entropy_CDB(wave_function, size_lst)


def Entropy_BDC(wave_function, size_lst):
    return Entropy_CDB(wave_function, size_lst)


def Entropy_DBC(wave_function, size_lst):
    return Entropy_CDB(wave_function, size_lst)


def Entropy_DCB(wave_function, size_lst):
    return Entropy_CDB(wave_function, size_lst)

# Creation of Probabilistic Hamiltonian

In [ ]:
def mean_squared_second_moment(lst, prob_lst):
    assert len(lst) == len(prob_lst)
    assert round(sum(prob_lst),5) == 1.0
    mean = 0
    sec_mom = 0
    for i in range(len(lst)):
        mean += lst[i] * prob_lst[i]
        sec_mom += lst[i]**2 * prob_lst[i]
        
    return mean**2, sec_mom

def try_to_get_distr( fraction = 5 ):
    
    amount_number = int(2)
    number_lst = [random.random() * 10 + 0.1 for i in range(amount_number)]
    prob_lst_unnormed = [random.random() for i in range(amount_number)]
    prob_lst = [ elem / sum(prob_lst_unnormed) for elem in prob_lst_unnormed]
    
    mean2, moment2 = mean_squared_second_moment(number_lst, prob_lst)
    
    if fraction + 0.5 >= (moment2 / mean2) >= fraction - 0.5:
        return (number_lst, prob_lst)
    
    else:
        return

def generate_prob_distribution( fraction = 5 , length = 100):
    amount_lst = []
    prob_lst = []

    while len(prob_lst) < length:
        a = try_to_get_distr( fraction )
        if a != None:
            amount_lst.append(a[0])
            prob_lst.append(a[1])

    prob_lst = np.array(prob_lst)
    prob_lst = prob_lst.flatten()
    prob_lst = [elem / np.sum(prob_lst) for elem in prob_lst]
    amount_lst = np.array(amount_lst).flatten()

    mean2, mom2 = mean_squared_second_moment(amount_lst, prob_lst)
#     assert mom2 / mean2 >= fraction
#     print('Mom2 / mean2 is ', mom2/ mean2)
    
    return amount_lst, prob_lst

def get_random_index( prob_cum ):
    '''This function generates an index randomnly drawn for a certain discrete probability distribution'''
    assert round(prob_cum[-1] ,5) == 1.0

    #Generate a random number between 0 and 1 and check where it would be in the cumulative distribution
    random_prob = random.random()
    index = bisect.bisect( prob_cum, random_prob )
    
    return index

def supersymmetric_Hamiltonian_mom2(amount_lst, prob_lst):
    '''This function generates the Hamiltonian in the second quantizes version and it takes into account the 
    random values lambda_i that we multiply with some of the terms. It outputs the jordan wigner transposed Hamiltonian
    
    It takes as an input the width of the 
    interval from which we will randomly draw the values for labda, and it takes a list with the positions where
    we want lambda not equal to 1. The list has values 0 or 1 where 0 indicates we do not take a random lambda and 
    1 indicates we do'''
    
    random_lambda_lst = []
    
#     #Assert that the position list has length N_lattice and it consists of integers 0 or 1
#     assert len(lambda_pos) == N_lattice
    
#     for elem in lambda_pos:
#         assert type(elem) == int
#         assert (elem == 0 or elem == 1)
        
    #Create empty operator
    Q_dagger = FermionOperator()

    #Cumulative probability is 
    prob_cumu = [sum(prob_lst[:i+1]) for i in range(len(prob_lst))]
    
    #Now we create our Operator Q    
    for i in range(N_lattice):
        random_lambda = amount_lst[get_random_index( prob_cumu )]
        random_lambda_lst.append(random_lambda)
        Q_dagger += random_lambda * Q_dagger_term(i)

    #We take the Hermitian Conjugate of this operator
    Q = hermitian_conjugated(Q_dagger)  

    #This will be our Hamiltonian
    Hamiltonian = Q * Q_dagger + Q_dagger * Q  

    #We take the jordan-wigner transformation
    Hamiltonian_jw = jordan_wigner(Hamiltonian)  

    #Return the jordan-wigner transformed Hamiltonian
    return Hamiltonian_jw, random_lambda_lst

def supersymmetric_Hamiltonian_gauss(mu = 0, sigma = 1):
    '''This function generates the Hamiltonian in the second quantizes version and it takes into account the 
    random values lambda_i that we multiply with some of the terms. It outputs the jordan wigner transposed Hamiltonian
    
    It takes as an input the width of the 
    interval from which we will randomly draw the values for labda, and it takes a list with the positions where
    we want lambda not equal to 1. The list has values 0 or 1 where 0 indicates we do not take a random lambda and 
    1 indicates we do'''
    
    random_lambda_lst = []
    
#     #Assert that the position list has length N_lattice and it consists of integers 0 or 1
#     assert len(lambda_pos) == N_lattice
    
#     for elem in lambda_pos:
#         assert type(elem) == int
#         assert (elem == 0 or elem == 1)
        
    #Create empty operator
    Q_dagger = FermionOperator()
    
    #Now we create our Operator Q    
    for i in range(N_lattice):
        random_lambda = random.gauss(mu, sigma)
        while round(random_lambda, 6) == 0:
            random_lambda = random.gauss(mu, sigma)
        
        random_lambda_lst.append(random_lambda)
        Q_dagger += random_lambda * Q_dagger_term(i)

    #We take the Hermitian Conjugate of this operator
    Q = hermitian_conjugated(Q_dagger)  

    #This will be our Hamiltonian
    Hamiltonian = Q * Q_dagger + Q_dagger * Q  

    #We take the jordan-wigner transformation
    Hamiltonian_jw = jordan_wigner(Hamiltonian)  

    #Return the jordan-wigner transformed Hamiltonian
    return Hamiltonian_jw, random_lambda_lst

def supersymmetric_Hamiltonian_gauss_abs(mu = 0, sigma = 1):
    '''This function generates the Hamiltonian in the second quantizes version and it takes into account the 
    random values lambda_i that we multiply with some of the terms. It outputs the jordan wigner transposed Hamiltonian
    
    It takes as an input the width of the 
    interval from which we will randomly draw the values for labda, and it takes a list with the positions where
    we want lambda not equal to 1. The list has values 0 or 1 where 0 indicates we do not take a random lambda and 
    1 indicates we do'''
    
    random_lambda_lst = []
           
    #Create empty operator
    Q_dagger = FermionOperator()
    
    #Now we create our Operator Q    
    for i in range(N_lattice):
        random_lambda = random.gauss(mu, sigma)
        if random_lambda > 0:
            random_lambda = random_lambda
        else:
            random_lambda = 0.1
        
        random_lambda_lst.append(random_lambda)
        Q_dagger += random_lambda * Q_dagger_term(i)

    #We take the Hermitian Conjugate of this operator
    Q = hermitian_conjugated(Q_dagger)  

    #This will be our Hamiltonian
    Hamiltonian = Q * Q_dagger + Q_dagger * Q  

    #We take the jordan-wigner transformation
    Hamiltonian_jw = jordan_wigner(Hamiltonian)  

    #Return the jordan-wigner transformed Hamiltonian
    return Hamiltonian_jw, random_lambda_lst

# Calculation of the mutual information

In [ ]:
def mut_info(wave_funtion , size_lst , part1 , part2):
    '''In this function you just say which parts of the final state you want the mutual information
    of. e.g. mut_inf(R,DB) returns the mutual information I(R:BD)'''
    
    assert type(part1) == str
    assert type(part2) == str
    
    Entropy_part1 = eval('Entropy_'+part1)(wave_funtion, size_lst)
    Entropy_part2 = eval('Entropy_'+part2)(wave_funtion, size_lst)
    Entropy_part1part2 = eval('Entropy_'+part1+part2)(wave_funtion, size_lst)
    
    mut_info = Entropy_part1 + Entropy_part2 - Entropy_part1part2
#     print(f'EntropyR is {Entropy_part1}')
#     print(f'EntropyBD is {Entropy_part2}')
#     print(f'EntropyRBD is {Entropy_part1part2}')

    assert np.isreal(mut_info)
    mut_inf = np.real(mut_info)
        
#     print(f'Mutual information I(R:BD) = {mut_inf}')
    
    return mut_info

def mut_inf_RBD(wave_func, size_lst):
    '''This function also gets the wave function for the specific mutual 
    information I(R:BD) from the backend cheat'''    
    
    part1 = 'R'
    part2 = 'BD'
    
    #Note that we always reverse our qubits in the entropy function so we never worry about doing this
    wave_func = cheat_order_qubits()
    
    #print(wave_func)
    info = mut_info( wave_func, size_lst, part1, part2) 
    
    return info



# List Manipulation

In [ ]:

def aver_std_lst(lst_lst):
    '''This function takes as input a list of lists and outputs the average and the standard deviation'''

    #Check if all the lists are the same size
    for lst in lst_lst:
        assert len(lst) == len(lst_lst[0])

    #Make numpy array
    lst_lst = np.array(lst_lst)

    #Take the average (the zip(*) brings all the elements having the same position into a list)
    aver_lst = [np.sum(elem) / len(elem) for elem in zip(*lst_lst)]

    #Compute the standard deviation sqrt(1/N sum( (x_ave - x)**2 ) )
    std_lst = [
        np.sqrt(np.sum(((np.sum(elem) / len(elem)) - elem)**2) / len(elem))
        for elem in zip(*lst_lst)
    ]

    return aver_lst, std_lst

def plot_MBL_AL(data_points = [], MBL = [], AL = [], begin = 0, end = 10):
    plt.semilogx(data_points[begin:end], MBL[begin:end])
    plt.semilogx(data_points[begin:end], AL[begin:end])
    plt.gca().legend(('MBL regime','Anderson regime'))
    plt.show()

# Entropy Time Evolutions

In [ ]:
def entanglement_entropy_time_evolution_XXZ(data_points,
                                            State_string_it=State_string,
                                            tus_stap=1,
                                            N_average=10,
                                            spin_operator=1,
                                            spin_z_operator=1,
                                            field_strength=5):
    '''Quantum Algortihm! This function takes as input the variables of the Hamiltonian, the datapoints and how often it averages
    to get a good average hamiltonian simulation. It outputs a list with the average entropy and the standard deviation. 
    The tus_stap means how many steps we want to do between the data_points to keep the error low. We will only output
    the values of the entropy at the data points not at the tus_stap'''

    #Set up empty lists that will consist of lists
    entropy_values_lst = []
    wave_functions_lst = []

    for i in tqdm(range(N_average), leave=False):

        #Set up the qubits
        q = eng.allocate_qureg(N_lattice)

        #Bring the qubits into the desired state
        String_to_State(State_string_it, q)

        #Set up the lists with the values
        entropy_values = []
        wave_functions = []

        #Calculate entropy by cheating
        entropy = entanglement_entropy(2**4, 2**4)
        wave_function = cheat_order_qubits()

        #Save entropy and wave function
        entropy_values.append(entropy)
        wave_functions.append(wave_function)

        #Create a list with the difference in time between the two timesteps
        dat_differ = [
            data_points[i + 1] - data_points[i]
            for i in range(len(data_points) - 1)
        ]

        #Create the Hamiltonian, note that we do not want to create a new hamiltonian for every timestep!
        hamiltonian = XXZ_Hamiltonian(spin_operator, spin_z_operator,
                                      field_strength)

        #Generate a list of operators that time evolve the system by the desired timestep
        ham_list = []
        for timestep in dat_differ:
            ham_step = trot(hamiltonian, evolution_time=timestep / tus_stap)
            ham_list.append(list(ham_step))

        #Now we do the time evolution and calculate the entropy after every timestep.
        #Iterate over every time evolution operator
        for i in tqdm(range(len(ham_list)), leave=False):

            #We time evolve the determined amount of steps in between
            with Loop(eng, tus_stap):
                #Time evolve
                time_evolve(q, ham_list[i])

            #Calculate entropy by cheating and add to the list
            entropy = entanglement_entropy(2**4, 2**4)
            entropy_values.append(entropy)

            #Get the wave function and add to the list
            wave_function = cheat_order_qubits()
            wave_functions.append(wave_function)

        #Save the list of values to the total list
        entropy_values_lst.append(entropy_values)
        wave_functions_lst.append(wave_functions)

        #Measure and delete the qubits
        All(Measure) | q
        del q

    #Take the average and standard deviation of the entropy values
    entropy_ave, entropy_std = aver_std_lst(entropy_values_lst)

    #Taking the average over the wave functions is not physical
    #(e.g. [(1,0) , (0,1)] averages to (0.5,0.5) and this is not normalized)

    return entropy_ave, entropy_std

def clas_sim_XXZ(ts, state, N_average = 5, a=1,b=0.2,c=5):
    '''Outputs list_sum (averaged list) and all the lists with the renyi entropies
    CLASSICAL SIMULATION'''
    renyi_list = [ ]
    
    dif_ts = [ts[i+1] - ts[i] for i in range(len(ts)-1)]

    for i in tqdm(range(N_average), leave = False):
        renyis = []
        
        v0 = np.array( String_to_vector(state) )
        v0 = reverse_qubits(v0)
        
        v1 =  v0.reshape(2**4, 2**4)
        rho = v1 @ v1.conjugate().transpose()
        renyi = -np.log2( np.trace( rho @ rho ) )
        assert np.isreal(renyi)
        renyis.append(np.real(renyi))

        #Create the Hamiltonian, note that we do not want to create a new hamiltonian for every timestep!
        hamiltonian = XXZ_Hamiltonian(a,b,c)

        for time_step in tqdm(dif_ts, leave = False):
            v0 = scipy.sparse.linalg.expm(-1j * get_sparse_operator(hamiltonian) * time_step) @ v0
            v1 =  v0.reshape(2**4, 2**4)
            rho = v1 @ v1.conjugate().transpose()
            renyi = -np.log2( np.trace( rho @ rho ) )
            assert np.isreal(renyi)
            renyis.append(np.real(renyi))
               
        renyi_list.append(renyis)

    list_sum, list_std = aver_std_lst(renyi_list)

    #Save plot and data
#     fig = plt.figure()
#     plt.ylim([0,2.5])
#     plt.semilogx(ts, list_sum)
#     plt.tight_layout()
    # plt.xlabel(f'Classical simulation of the evolution of the Second Renyi entropy \n through time with the interval {width_interval} of the state '+State_string)
    # plt.savefig(f'{dirs}/Classical_PlotState_{name}_Lambda_{lambda_string}_steps{str_timesteps}_rand_interval{width_interval}_LambdaPeriod{lambda_period}_LambdaStart{lambda_start}_Averaged{N_simulations}.png')
#     plt.show

    return list_sum, renyi_list

def entanglement_entropy_time_evolution_gauss_abs(
        data_points,
        State_string_it=State_string,
        tus_stap=1,
        N_average=10,
        width_interval=0,
        lambda_pos=[0 for i in range(N_lattice)]):
    '''QUANTUM ALGORITHM! This function takes as an input the different configurations of the random factors which 
    we multiply in the Hamiltonian. It outputs a list with the average entropy and its standard deviation
    dependend on how many times we average (given by N_average). 
    The tus_stap means how many steps we want to do between the data points to keep the error of the trotterization
    scheme low. Only the values of the entanglement entropy at the datapoints will be in the output. Uses gaussian random 
    random variables as inputs for the states. Negative values are cast to 0.1.'''

    #Set up empty lists that will consist of lists
    entropy_values_lst = []
    wave_functions_lst = []

    for i in tqdm(range(N_average), leave=False):

        #Set up the qubits
        q = eng.allocate_qureg(N_lattice)

        #Bring the qubits into the desired state
        String_to_State(State_string_it, q)

        #Set up the lists with the values
        entropy_values = []
        wave_functions = []

        #Calculate entropy by cheating
        entropy = entanglement_entropy(2**4, 2**4)
        wave_function = cheat_order_qubits()

        #Save entropy and wave function
        entropy_values.append(entropy)
        wave_functions.append(wave_function)

        #Create a list with the difference in time between the two timesteps
        dat_differ = [
            data_points[i + 1] - data_points[i]
            for i in range(len(data_points) - 1)
        ]

        #Create the Hamiltonian, note that we do not want to create a new hamiltonian for every timestep!
        amount_lst, prob_lst = generate_prob_distribution(length = 200)
        
        hamiltonian = supersymmetric_Hamiltonian_gauss_abs( amount_lst, prob_lst )

        #Generate a list of operators that time evolve the system by the desired timestep
        ham_list = []
        for timestep in dat_differ:
            ham_step = trot(hamiltonian, evolution_time=timestep / tus_stap)
            ham_list.append(list(ham_step))

        #Now we do the time evolution and calculate the entropy after every timestep.
        #Iterate over every time evolution operator
        for i in tqdm(range(len(ham_list)), leave=False):

            #We time evolve the determined amount of steps in between
            with Loop(eng, tus_stap):
                #Time evolve
                time_evolve(q, ham_list[i])

            #Calculate entropy by cheating and add to the list
            entropy = entanglement_entropy(2**4, 2**4)
            entropy_values.append(entropy)

            #Get the wave function and add to the list
            wave_function = cheat_order_qubits()
            wave_functions.append(wave_function)

        #Save the list of values to the total list
        entropy_values_lst.append(entropy_values)
        wave_functions_lst.append(wave_functions)

        #Measure and delete the qubits
        All(Measure) | q
        del q

    #Take the average and standard deviation of the entropy values
    entropy_ave, entropy_std = aver_std_lst(entropy_values_lst)

    #Taking the average over the wave functions is not physical
    #(e.g. [(1,0) , (0,1)] averages to (0.5,0.5) and this is not normalized)

    return entropy_ave, entropy_std

# Main Average States

In [ ]:
renyi_lst = []
random_lam_lst = []
random_states = []
mean_lst = []
frac_lst = [2]

random.seed(12)

for frac in tqdm(frac_lst):
    for i in tqdm(range(20)):
        random_state = random_supersym_state()
        random_states.append(random_state)
        v0 = np.array(String_to_vector(random_state))
        v0 = reverse_qubits(v0)
        ts = 10**np.linspace(-3,7, 100)
        renyis = []
        dif_ts = [ts[i+1] - ts[i] for i in range(len(ts)-1)]

        v1 =  v0.reshape(2**4, 2**4)
        rho = v1 @ v1.conjugate().transpose()
        renyi = -np.log2( np.trace( rho @ rho ) )
        assert np.isreal(renyi)
        renyis.append(np.real(renyi))

        #Create the Hamiltonian, note that we do not want to create a new hamiltonian for every timestep!
        hamiltonian, random_lam = supersymmetric_Hamiltonian_gauss( 0, frac )
        oper_ham = get_sparse_operator(hamiltonian)

        for time_step in tqdm(dif_ts, leave = False):
            v0 = scipy.sparse.linalg.expm(-1j * oper_ham * time_step) @ v0
#             v0 = scipy.sparse.linalg.expm_multiply(-1j * oper_ham * time_step, v0)
            v1 =  v0.reshape(2**4, 2**4)
            rho = v1 @ v1.conjugate().transpose()
            renyi = -np.log2( np.trace( rho @ rho ) )
            assert np.isreal(renyi)
            renyis.append(np.real(renyi))

        renyi_lst.append(renyis)
        random_lam_lst.append(random_lam)

    aver_ren, b = aver_std_lst(renyi_lst)

    plt.semilogx(ts, aver_ren)

In [ ]:
random.seed(12)

def entropy_evolution_Gauss(mu, sigma, absolute = False):
    '''This is a simulation that uses Gaussian values where negative values are cast to 0.1.'''
    renyi_lst = []
    random_states = []
    random_lam_lst = []
    ave_lst_lst = []
    vector_lst = []
    ave_lst = []
    
    for ave in tqdm(range(3), leave = False):
        
        random_state = random_supersym_state(3)
        random_states.append(random_state)
        for i in tqdm(range(20), leave = False):
            
            vector = []
            v0 = np.array(String_to_vector(random_state))
            v0 = reverse_qubits(v0)
            vector.append(v0)
            ts = 10**np.linspace(-3,2.5, 60)
            renyis = []
            dif_ts = [ts[i+1] - ts[i] for i in range(len(ts)-1)]

            v1 =  v0.reshape(2**4, 2**4)
            rho = v1 @ v1.conjugate().transpose()
            renyi = -np.log2( np.trace( rho @ rho ) )
            assert np.isreal(renyi)
            renyis.append(np.real(renyi))

            #Create the Hamiltonian, note that we do not want to create a new hamiltonian for every timestep!
            if absolute == True:
                hamiltonian, random_lam = supersymmetric_Hamiltonian_gauss_abs( mu , sigma )
            else:
                hamiltonian, random_lam = supersymmetric_Hamiltonian_gauss( mu , sigma )
            
            oper_ham = get_sparse_operator(hamiltonian)

            for time_step in tqdm(dif_ts, leave = False):
#                 v0 = scipy.sparse.linalg.expm(-1j * oper_ham * time_step) @ v0
                v0 = scipy.sparse.linalg.expm_multiply(-1j * oper_ham * time_step, v0)
                v1 =  v0.reshape(2**4, 2**4)
                rho = v1 @ v1.conjugate().transpose()
                renyi = -np.log2( np.trace( rho @ rho ) )
                assert np.isreal(renyi)
                renyis.append(np.real(renyi))
                vector.append(v0)
                
            vector_lst.append(vector)
            renyi_lst.append(renyis)
            ave_lst.append(renyis)
            random_lam_lst.append(random_lam)

        ave_lst.append(aver_std_lst(ave_lst)[0])
        
        
    ave_lst_lst.append(aver_std_lst(ave_lst)[0])
    plt.semilogx(ts, aver_std_lst(ave_lst)[0])
    
    return renyi_lst, random_states, ave_lst_lst, vector_lst, random_lam_lst
    

In [ ]:
def vector_to_state(vector):
    '''This function takes a vector as input and outputs the corresponding value in terms of states'''

    len_vec = len(vector)

    #transform vector to numpy array for easier use of numpy functions
    vector = np.array(vector)

    #Check normalization
    assert round(np.linalg.norm(vector),4) == 1.000

    #Check the vector is a power of 2, and determine number of particles
    assert round(np.log2(len(vector)), 5) == int(np.log2(len(vector)))
    num_part = int(np.log2(len(vector)))

    #Note that the vector is reversed in our perspective
    vector = reverse_qubits(vector)

    q = eng.allocate_qureg(num_part)

    StatePreparation(vector) | q
    eng.flush()
    p0_lst = []
    p1_lst = []

    for i in range(num_part):
        p0 = eng.backend.get_probability('0', [q[i]])
        p1 = eng.backend.get_probability('1', [q[i]])
        p0_lst.append(p0)
        p1_lst.append(p1)

    All(Measure) | q
    del q

    return p1_lst

In [ ]:
data_points = 10**np.linspace(-3, 5, 60)

# Set up formatting for the movie files
FFwriter = animation.FFMpegWriter(fps=5, codec="libx264")  

#Set up the animation
data = probs_states9[2]
number_of_frames = len(data_points)
data2 = fi[1][2][0]
objects = [str(i) for i in range(N_lattice)]

fig = plt.figure()
# fig.suptitle(f'Starting state is {state}, the lambda are localized at \n {lam_pos} and chosen from the interval {1-width_int, 1 + width_int}. \n Plotted above is the probability of a particle being at a site \n at each timestep. Below the entanglement entropy is plotted \n and the red dot indicates the time.', y = 1.2)
p1 = fig.add_subplot(2,1,1)
p2 = fig.add_subplot(2,1,2)

def update_graph(num, data, data2):
    p1.cla()
    p1.set_ylim([0.0,1.0])
    p1.bar(objects, data[num], align = 'center', alpha=0.5)

    p2.cla()
    p2.semilogx(data_points, data2)
    p2.set_ylim([0,3])
    p2.semilogx( data_points[num],data2[num], 'ro')

plt.cla()
ani = animation.FuncAnimation(fig, update_graph, number_of_frames, fargs=(data, data2 ) )
plt.cla()

HTML(ani.to_jshtml())